In [ ]:
from azure.identity import DefaultAzureCredential, ClientSecretCredential
from azure.mgmt.storage import StorageManagementClient
from azure.storage.blob import BlobServiceClient
from azure.mgmt.resource import ResourceManagementClient, SubscriptionClient

import pandas as pd
import re, os

from azure.mgmt.authorization import AuthorizationManagementClient
from azure.mgmt.authorization.models import RoleAssignmentCreateParameters
from azure.core.exceptions import ResourceExistsError

from dotenv import load_dotenv
load_dotenv()




In [ ]:
def get_credeantials():
    #import json

    #with open('config.json') as config_file:
        #config = json.load(config_file)

    # tenant_id = config['tenant_id']
    # client_id = config['client_id']
    # client_secret = config['client_secret']

    
    TENANT_ID = os.getenv('TENANT_ID')
    CLIENT_ID = os.getenv('CLIENT_ID')
    CLIENT_SECRET = os.getenv('CLIENT_SECRET') 

    credentials = ClientSecretCredential(TENANT_ID, CLIENT_ID, CLIENT_SECRET)
    return credentials

In [ ]:
def get_subscriptions():
    # Create the SubscriptionClient object using Azure AD credentials
    credential = get_credeantials() #DefaultAzureCredential()
    #credential = DefaultAzureCredential() #DefaultAzureCredential()
    subscription_client = SubscriptionClient(credential)
    
    # List all subscriptions
    subscriptions = subscription_client.subscriptions.list()
    
    # Collect subscription details in a list
    subscription_list = []
    for subscription in subscriptions:
        subscription_list.append({   
            "subscription_name": subscription.display_name,
            "subscription_id": subscription.subscription_id,
        })
    df = pd.DataFrame(subscription_list)

    return df

In [ ]:
def get_resources(subscription_id):
    credential = get_credeantials() #DefaultAzureCredential()
    #credential = DefaultAzureCredential() #DefaultAzureCredential()
    resource_client = ResourceManagementClient(credential, subscription_id)

    resources = resource_client.resources.list()

    resource_list = []

    for resource in resources:
        #print(f"Name: {resource.name}, Type: {resource.type}, Location: {resource.location}")
        resource_list.append({
            'subscription_id': subscription_id,
            'Resource Name': resource.name,
            'Type': re.sub("Microsoft.", "", resource.type, flags=re.IGNORECASE), #resource.type.replace("Microsoft.", ""),
            'Location': resource.location
        })
    df = pd.DataFrame(resource_list)
    return df






In [5]:
def list_storage_accounts(subscription_id):
    credential = DefaultAzureCredential()
    storage_client = StorageManagementClient(credential, subscription_id)
    storage_accounts = storage_client.storage_accounts.list()
    return storage_accounts

In [6]:
def list_files_with_sizes(storage_account_name, container_name):
    # Create the BlobServiceClient object using Azure AD credentials
    credential = DefaultAzureCredential()  

    #blob_service_client = BlobServiceClient(account_url=f"https://{storage_account_name}.blob.core.windows.net", credential=SAS_TOKEN)
    blob_service_client = BlobServiceClient(account_url=f"https://{storage_account_name}.blob.core.windows.net", credential=credential)
    
    # Get the container client
    container_client = blob_service_client.get_container_client(container_name)
    
    # List the blobs in the container
    blob_list = container_client.list_blobs()
    #print(f"**{blob_list}**")
    # Print the name and size of each blob
    for blob in blob_list:
        print(f"Name: {blob.name}, Size: {blob.size/1024/1024:.1f} MB")
        #print(f"\t{blob}")

In [7]:
def list_containers(storage_account_name):
    # Create the BlobServiceClient object using Azure AD credentials
    credential = DefaultAzureCredential()
    blob_service_client = BlobServiceClient(account_url=f"https://{storage_account_name}.blob.core.windows.net", credential=credential)
    
    # List the containers in the storage account
    containers = blob_service_client.list_containers()

    return containers

In [8]:
def assign_role_to_subscription(subscription_id, role_definition_id, principal_id):
    # Create the AuthorizationManagementClient object using Azure AD credentials
    credential = DefaultAzureCredential()
    auth_client = AuthorizationManagementClient(credential, subscription_id)
    
    # Define the scope (subscription level)
    scope = f"/subscriptions/{subscription_id}"
    
    # Generate a unique role assignment name
    import uuid
    role_assignment_name = str(uuid.uuid4())
    
    # Create the role assignment parameters
    role_assignment_params = RoleAssignmentCreateParameters(
        role_definition_id=role_definition_id,
        principal_id=principal_id
    )
    
    # Assign the role
    role_assignment = auth_client.role_assignments.create(
        scope=scope,
        role_assignment_name=role_assignment_name,
        parameters=role_assignment_params
    )
    
    return role_assignment